In [1]:
# Import Libraries

import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime, timedelta, date
from pmdarima.arima import auto_arima
from fbprophet import Prophet

In [31]:
# Create DF to Fill Empty Dates - Não Precisa

def set_hour(date1):
  day1 = datetime(date1.year, date1.month, date1.day,0,0,0)
  return day1

dftime = pd.DataFrame(pd.period_range(start=datetime(2019,11,4,0,0,0), periods=24, freq='7D1H', name = 'Date1'))
dftime['date'] = dftime['Date1'].apply(lambda x: set_hour(x))
dftime = dftime.drop('Date1', 1)
dftime.set_index('date', inplace = True)

In [127]:
# Connection data to Database

PATH = '/Users/renato/Downloads/Crystalball/'
DATABASE_MODEL = 'modelsample.db'
MODEL_CSV = 'model.csv'
DATABASE_HISTORY = 'history.db'
DATABASE_CACHE = 'cache.db'
USER = 'user'
PASSWORD = 'password'
PORT = '5432'
HOST = '127.0.0.1'

In [33]:
# Default Data

CPF = '14566178854'

In [34]:
def get_data(cpf):
    try:
        connection = PATH + DATABASE_MODEL
        sqliteConnection = sqlite3.connect(connection)
        cursor = sqliteConnection.cursor()
        sqlite_select_query = """SELECT * from modelsample where cpf = """ + cpf
        cursor.execute(sqlite_select_query)
        records = cursor.fetchall()
        records = pd.DataFrame(records)
        records.columns = ['cpf', 'date1', 'prod_id', 'qtde']
        records['date'] =  pd.to_datetime(records['date1'],  format='%Y-%m-%d' )
        records = records.drop('date1', 1)

        cursor.close()
        
        return(records)

    except sqlite3.Error as error:
        print("Failed to read data from sqlite table", error)
    finally:
        if (sqliteConnection):
            sqliteConnection.close()

In [134]:
# Feature Engineering

def feature_engineering(df):

    # DataFrame for Loop

    dfloop = df.groupby(['prod_id'], as_index=False)['qtde'].count()
    dfloop

    data = pd.DataFrame()
    data['qtde'] = 1
    data['lag1'] = 2
    data['lag2'] = 3
    data['lag3'] = 4
    data['lag4'] = 5
    data['lag5'] = 6
    data['lag6'] = 7
    data['lag7'] = 8
    data['lag8'] = 9
                
    
    #Iteract for product
    for i, r in dfloop.iterrows():
        # Filtrar Item_id em um novo DF
        dffilter = df[df['prod_id'] == r['prod_id']]

        # merge data
        dffilter['date1'] = dffilter['date']
        dffilter.set_index('date', inplace = True)
        dfmerge = pd.concat([dffilter,dftime], axis=1)
        dfmerge['date1'] = dfmerge.index
        dfmerge['cpf'] = CPF
        dfmerge['prod_id'] = r['prod_id']
        dfmerge = dfmerge.fillna(0)

        
        #Colocar lags
        dfmerge['lag1'] = dfmerge.qtde.shift(1)
        dfmerge['lag2'] = dfmerge.qtde.shift(2)
        dfmerge['lag3'] = dfmerge.qtde.shift(3)
        dfmerge['lag4'] = dfmerge.qtde.shift(4)
        dfmerge['lag5'] = dfmerge.qtde.shift(5)
        dfmerge['lag6'] = dfmerge.qtde.shift(6)
        dfmerge['lag7'] = dfmerge.qtde.shift(7)
        dfmerge['lag8'] = dfmerge.qtde.shift(8)
        dfmerge = dfmerge.dropna()

        
        #Drop Columns
        dfmerge = dfmerge.drop('cpf', 1)

        # Salvar previsão em DF
        data = data.append(dfmerge)

    #Drop Columns
    data = data.drop('prod_id', 1)
    data = data.drop('date1', 1)


    return data

In [135]:
def save_data(df):
    
    file = PATH+MODEL_CSV
    print(file)
    
    print(type(df))
    
    df.to_csv(PATH+MODEL_CSV, index = False, header=True)

    return True

    

In [136]:

df = get_data (CPF)
engdata = feature_engineering(df)
retorno = save_data(engdata)
#retorno

/Volumes/Dados/Users/renato/anaconda3/envs/dev_phyton/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



/Users/renato/Downloads/Crystalball/model.csv
<class 'pandas.core.frame.DataFrame'>
